In [ ]:
Data Exploration and Cleanup Process

In [ ]:
# Importing Dependencies
import matplotlib.pyplot as plt
import requests
import pandas as pd
import os
import json
import numpy as np
import seaborn as sb
import time

#load your API keys
file_name ="C:\\Users\\spaug\\Python\\KU_Coding\\kubootcamp0\\api_keys.json"
data = json.load(open(file_name))
o_key = 'vvr6c9yegfkzkudyzvvg5za8'

We started by seeing what the JSON data looked like, and 
grabbed a unique identifier for each NFL team. We exported  this list into 'teams_df.csv'.

In [ ]:
# Setting up URL and API call for team_id
url = "http://api.sportradar.us/nfl-ot2/seasontd/"
season = "REG"
year = "2017"
output_format = "json"
team_id = "33405046-04ee-4058-a950-d606f8c30852"

query_url = url + year + "/" + season + "/teams/" + team_id + "/statistics." + output_format + "?api_key=" + o_key
nfl_response = requests.get(query_url)
nfl_json = nfl_response.json()

team_id_url = "http://api.sportradar.us/nfl-ot2/league/hierarchy.json?api_key="

team_id_response = requests.get(team_id_url + o_key)
team_id_json = team_id_response.json()

# Setting up teams_df variables
conf_count = 0
conferences = []
divisions = []
teams = []
team_id = []
venue_name = []
venue_city = []
venue_state = []
venue_capacity = []
venue_surface = []
venue_roof_type = []

# Starting conference gathering loop
for conference in team_id_json['conferences']:

    # Declaring division counter inside so that it resets for each iteration
    div_count = 0
    
    # Starting division gathering loop
    for division in team_id_json['conferences'][conf_count]:
        
        # Declaring teams counter inside so that it resets for each iteration
        team_count = 0
        
        # Starting team gathering loop
        for team in team_id_json['conferences'][conf_count]['divisions'][div_count]:

            # Creating variables for easier handling when appending
            temp_name = team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['name']
            temp_market = team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['market']
            
            # Appending conferences, divisions, team name and team IDs to lists
            conferences.append(team_id_json['conferences'][conf_count]['name'])
            divisions.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['name'])
            teams.append((str(temp_market) + " " + str(temp_name)))
            team_id.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['id'])
            venue_name.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['venue']['name'])
            venue_city.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['venue']['city'])
            venue_state.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['venue']['state'])
            venue_capacity.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['venue']['capacity'])
            venue_surface.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['venue']['surface'])
            venue_roof_type.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['venue']['roof_type'])
            
            # Incrementing teams counter
            team_count += 1
            
        # Incrementing division counter
        div_count += 1
    
    # Incrementing conference counter
    conf_count += 1
    

teams_df = pd.DataFrame(list(zip(teams, conferences, divisions, venue_name, venue_city, venue_state, venue_capacity, \
                                 venue_surface, venue_roof_type, team_id)), \
                                 columns=['Team', 'Conference', 'Division', 'Venue Name', 'Venue City', \
                                          'Venue State', 'Venue Capacity', 'Playing Surface', 'Stadium Type', 'Team ID'])

#export to teams_df, this file contains data on each NFL team and stadium, we used this to get TEAM_IDs
teams_df.to_csv('teams_df.csv')

We then looped through each team in teams_df, and retrieved seasonal stats 
for that team from each year the service had data from (2000-2017). 
We appended each teams data to a large data frame we called 'all_players.csv'. 
Here is where we ran into our first problem - the seasonal stats we downloaded 
did not include colleges for each player.

In [ ]:
all_teams_dict = {}
#loop through each teams data
# change range above for desired # of teams, to get full list use
for x in range(0, 33):
    team_id = teams_df['Team ID'][x]
    print(teams_df['Team'][x])
    team_dict = {}
    
#     loop throug each years data
    for y in range(0, 18):
#       change range above for desired of years, where 0,17 is the max range (2000 - 2017)
        year = y + 2000
        print(year)
        team_season_url = 'http://api.sportradar.us/nfl-ot2/seasontd/' + str(year) + '/reg/teams/' + team_id + '/statistics.json?api_key=' + o_key
        team_season_response = requests.get(team_season_url)
        time.sleep(1)
        response = team_season_response.json()
        team_dict[y] = response
    all_teams_dict[x] = team_dict
    
master_id = []
position = []
games_played = []
name = []
p_yards = []
p_td = []
p_int = []
ru_yards = []
ru_td = []
fumbles = []
rec_yards = []
rec_td = []
rec_rec = []
kick_ret = []
punt_ret = []
year_list = []

for x in all_teams_dict:
    for y in all_teams_dict[x]:
        for z in range(0,len(all_teams_dict[x][y]['players'])):
            try:
                master_id.append(all_teams_dict[x][y]['players'][z]['id'] + '_' + all_teams_dict[x][y]['id'])
            except KeyError:
                master_id.append(0)
            try:
                year_list.append(all_teams_dict[x][y]['season']['year'])
            except KeyError:
                year_list.append(0)
            try:    
                games_played.append(all_teams_dict[x][y]['players'][z]['games_played'])
            except KeyError:
                games_played.append(0)
            try:
                name.append(all_teams_dict[x][y]['players'][z]['name'])
            except KeyError:
                name.append(0)
            try:
                position.append(all_teams_dict[x][y]['players'][z]['position'])
            except KeyError:
                position.append(0)
            try:
                p_yards.append(all_teams_dict[x][y]['players'][z]['passing']['yards']  )
            except KeyError:
                p_yards.append(0)
            try:
                p_td.append(all_teams_dict[x][y]['players'][z]['passing']['touchdowns'])
            except KeyError:
                p_td.append(0)
            try:
                p_int.append(all_teams_dict[x][y]['players'][z]['passing']['interceptions'])
            except KeyError:
                p_int.append(0)
            try:
                ru_yards.append(all_teams_dict[x][y]['players'][z]['rushing']['yards'])
            except KeyError:
                ru_yards.append(0)
            try:
                ru_td.append(all_teams_dict[x][y]['players'][z]['rushing']['touchdowns']  )
            except KeyError:
                ru_td.append(0)
            try:
                fumbles.append(all_teams_dict[x][y]['players'][z]['fumbles']['fumbles']  )
            except KeyError:
                fumbles.append(0)
            try:
                rec_yards.append(all_teams_dict[x][y]['players'][z]['receiving']['yards']   )
            except KeyError:
                rec_yards.append(0)
            try:
                rec_td.append(all_teams_dict[x][y]['players'][z]['receiving']['touchdowns'] )
            except KeyError:
                rec_td.append(0)
            try:
                rec_rec.append(all_teams_dict[x][y]['players'][z]['receiving']['receptions'] )
            except KeyError:
                rec_rec.append(0)
            try:
                kick_ret.append(all_teams_dict[x][y]['players'][z]['kick_returns']['touchdowns'])
            except KeyError:
                kick_ret.append(0)
            try:
                punt_ret.append(all_teams_dict[x][y]['players'][z]['punt_returns']['touchdowns'])
            except KeyError:
                punt_ret.append(0)
                
players_df = pd.DataFrame(list(zip(master_id, name, year_list, position, 
                                    games_played, p_yards, p_td, p_int, ru_yards,
                                    ru_td, fumbles, rec_yards, rec_td, rec_rec, kick_ret, punt_ret)), \
                          columns=['Master_Id','Name', 'Year', 'Position','Games_Played','P_Yards','P_TD', 'P_Int',
                                 'Rush_Yards','Rush_TD','Fumbles','Rec_Yards','Rec_TD','Receptions',
                                  'Kick_Returns', 'Punt_Returns'])

players_df.to_csv('all_players.csv')

To remedy this, we had to filter the list of 35,000+ rows that contained multiple years of 
data for each player, and extract a list of unique player IDs. The total unique IDs 
were ~ 9,000, so we limited the list to offensive players only since they generate the most 
fantasy points. Surprisingly this was only about 2,600 records. We took this list and made a 
new call to a different Sportradar service for individual data. We then retrieved each college 
associated with a players uniqueID, and exported that to 'player_college.csv'. We did not obtain this file until late in the data collection process, so we adding it in was done last, and that part of the process is described below.

In [ ]:
file_name = 'all_players.csv'
all_players_df = pd.read_csv(file_name)

positions_list = ['QB','WR','TE','FB','RB']
offense_df = all_players_df.loc[all_players_df['Position'].isin(positions_list)]
offense_df1 = offense_df.reset_index()

player_list = []
team_list = []
for x in range(0,len(offense_df1)):
    player_list.append(offense_df1['Master_Id'].str.split('_')[x][0])
    team_list.append(offense_df1['Master_Id'].str.split('_')[x][1])
    print(str(x) + '/' + str(len(offense_df1)))
    
    
player_df1 = pd.DataFrame(player_list)
player_df1.columns = ['Player_Id']
team_df1 = pd.DataFrame(team_list)
team_df1.columns = ['Team_Id']
offense_df2 = pd.concat([player_df1, team_df1, offense_df1], axis=1)
offense_df3 = offense_df2.iloc[:,[0,1,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]]
offense_df3.to_csv
offense_df4 = offense_df3.drop_duplicates('Player_Id')
offense_df3.to_csv('Offensive_Only.csv')

offense_df5 = offense_df4.iloc[:,[0,2]]
offense_df5 = offense_df5.reset_index()
offense_df5 = offense_df5.iloc[:,[1,2]]
offense_df5.to_csv('unique_players.csv')

# Setting up API key pulls - Need 3 seperate keys to get around call limit
api_dir = os.path.dirname(os.path.dirname(os.path.realpath('__file__')))
file_name = os.path.join(api_dir, "api_keys.json")
data = json.load(open(file_name))

one_key = data['sportradar_nfl_official_key1']
two_key = data['sportradar_nfl_official_key2']
three_key = data['sportradar_nfl_official_key3']

# print(ce_key, ez_key, jz_key)

# Finding file with unique player names to search and then starting a loop to interate
player_file = "unique_players.csv"
with open(player_file, "r") as p_list:
    # Reading in all the player data
    player_df = pd.read_csv(p_list)

save_path = "unique_players.csv"

# Setting up URL and API call for team_id
key = [one_key, two_key, three_key]
counter = 0
manip_count = 0
college_df = []
#player_df['Player_Id']

for player in player_df['Player_Id']:
    # Building url and calling api
    url = f"https://api.sportradar.us/nfl-ot2/players/{player}/profile.json?api_key=" + str(key[counter])

    # Calling API - counts against API limit
    api_response = requests.get(url)
    
    # Hard coding in a 1 call per second limit - api server will reject anything faster 
    time.sleep(1.1)
    
    # Reading json
    api_json = api_response.json()
    
    # Counters - cycling keys every 900 calls so as to not hit a limit on any key
    manip_count += 1
    counter = manip_count % 3
    
    # Appending college, N/A if nothing
    try:
        college_df.append(api_json['college'])
        print(str(manip_count) + " - " + str(api_json['college']))
    except:
        college_df.append("N/A")
        print(str(manip_count) + " - N/A")
        
player_df = pd.merge(player_df, college_df, right_index=True, left_index=True)
player_df.rename(index=str, columns={"0": "College"})
player_df.columns=['blank', 'player_id', 'name', 'college']
player_df = player_df.iloc[:,[1,2,3]]
player_df.to_csv("player_college.csv")

The next step in our data collection process was to calculate the amount of fantasy points 
generated by each player. To do this we made a dictionary of each stat category (passing yards, 
rushing yards, etc..) and assigned a multiplier value for each one. We looped through each 
row in our offensive_players dataframe, multiplying each column by the appropriate value. 
This was an awesome script we called 'apply_fantasy.ipynb'. 

In [ ]:
CSV = "Offense_Only.csv"

with open(CSV, "r") as inbox:
    # DATA is the original data
    DATA = pandas.read_csv(inbox)

RENAME = {"P_Yards": "passing yards",
          "P_TD": "passing touchdowns", 
          "P_Int": "passing interceptions", 
          "Rush_Yards": "rushing yards",
          "Rush_TD": "rushing touchdowns",
          "Fumbles": "fumbles",
          "Rec_Yards": "receiving yards",
          "Rec_TD": "receiving touchdowns",
          "Receptions": "receptions",
          "Kick_Returns": "kick returns",
          "Punt_Returns": "punt returns"
         }    
result = DATA.rename(columns = RENAME)    
LOCATAION = "fantasy_stats_final.csv"

END_POINTS = {
    "passing yards": 0.04,
    "passing touchdowns": 4,
    "passing interceptions": -2,
    "rushing yards": 0.1,
    "rushing touchdowns": 6,
    "receiving yards": 0.1,
    "receiving touchdowns": 6,
    "fumbles": -2,
    "kick returns": 6,
    "punt returns": 6
}

# result = pandas.DataFrame()

for each in END_POINTS:
    result[each] = result[each].map(lambda x: x*END_POINTS[each])
result.drop(result.columns[0], axis = 1, inplace = True)
with open(LOCATAION, "w") as outbox:
    result.to_csv(outbox, index = False)
    
    
CSV = "fantasy_stats_final.csv"

with open(CSV, "r") as inbox:
    # DATA is the original data
    data = pandas.read_csv(inbox)
    
totals_list = []
for x in range(0,len(data)):  
    totals = data.iloc[x,[6,7,8,9,10,11,12,13,15,16]].sum()
    totals_list.append(totals)
totals_df = pd.DataFrame(totals_list)
data_df = pd.DataFrame(data)
totals_df.columns = ['total annual points']
final_with_totals = pd.concat([data_df, totals_df], axis=1)
final_with_totals.to_csv('final_with_totals.csv')

At this point we had 2 data sets, 1 contained all of the stats information, the other contained
all the college data for each player. Using a final script we merged the two datasets together.

In [ ]:
CSV = "final_with_totals.csv"

with open(CSV, "r") as inbox:
    # DATA is the original data
    data = pandas.read_csv(inbox)

merged = data.merge(data2, left_on=['Player_Id'], right_on=['player_id'], how='left')
merged.to_csv('actually_final_i_promise.csv')